In [ ]:
import google.generativeai as genai
from dotenv import load_dotenv
import os 
import pandas as pd
load_dotenv()
# YOUR API KEY
genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))

In [ ]:
genai.configure(transport='grpc')

In [ ]:
!gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

In [ ]:
# Load the Excel file
excel_file = 'news_summary.xlsx' 
df = pd.read_excel(excel_file)
training_data = []
for index, row in df.head(1000).iterrows():
    training_data.append({'text_input': str(row['ctext']), 'output': str(row['text'])})
    

In [ ]:
base_model = [
    m for m in genai.list_models()
    if "createTunedModel" in m.supported_generation_methods][0]
    
name = 'news-summ'

In [ ]:
operation = genai.create_tuned_model(
    source_model=base_model.name,
    training_data=training_data, 
    id=name,
    epoch_count=10,
    batch_size=10,
    learning_rate=1
)

In [ ]:
model = genai.get_tuned_model(f'tunedModels/{name}')
model


In [ ]:
model.state

In [ ]:
operation.metadata


In [ ]:
#check progress
import time

for status in operation.wait_bar():
  time.sleep(30)